In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from joblib import dump,load

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
df

,Formula,K_frac,Rb_frac,Cs_frac,MA_frac,FA_frac,Li_frac,Ca_frac,Sr_frac,Ba_frac,...,X_Heat_of_fusion(kJ/mol),A_Heat_of_vap(kJ/mol),B_Heat_of_vap(kJ/mol),X_Heat_of_vap(kJ/mol),A_Electronegativity,B_Electronegativity,X_Electronegativity,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,MAGeCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,334.30,10.20,2.55,2.01,3.16,-0.391,1.954,0.148
1,MAGeBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,334.30,14.73,2.55,2.01,2.96,-0.243,1.612,0.204
2,MAGeI_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7.76,23.85,334.30,20.90,2.55,2.01,2.66,-0.030,1.311,0.231
3,MASnCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,290.37,10.20,2.55,1.96,3.16,-0.246,1.582,0.165
4,MASnBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,290.37,14.73,2.55,1.96,2.96,-0.228,1.262,0.198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,FAGeI_1.125_Cl_1.875,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.92,29.00,334.30,14.21,2.57,2.01,2.97,-1.075,2.156,0.110
484,FASnI_0.375_Cl_2.625,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.78,29.00,290.37,11.54,2.57,1.96,3.10,-1.133,2.576,0.062
485,MAPbCl_1.125_I_1.875,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,6.05,23.85,177.90,16.89,2.55,2.33,2.85,-0.003,1.867,0.164
486,MASrCl_0.75_I_2.25,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,6.62,23.85,136.90,18.23,2.55,0.95,2.79,0.101,3.693,0.001


In [3]:
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
Y

,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,-0.391,1.954,0.148
1,-0.243,1.612,0.204
2,-0.030,1.311,0.231
3,-0.246,1.582,0.165
4,-0.228,1.262,0.198
...,...,...,...
483,-1.075,2.156,0.110
484,-1.133,2.576,0.062
485,-0.003,1.867,0.164
486,0.101,3.693,0.001


In [4]:
Y=Y['Band Gap (eV)']
Y

0      1.954
1      1.612
2      1.311
3      1.582
4      1.262
       ...  
483    2.156
484    2.576
485    1.867
486    3.693
487    3.591
Name: Band Gap (eV), Length: 488, dtype: float64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 57) ; y_train's shape is (390,)
X_test's shape is (98, 57) ; y_test's shape is (98,)


In [6]:
##gridsearch and cv select model##
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # 隐藏层神经元数量和层数的组合
    'activation': ['relu', 'tanh'],  # 激活函数选项
    'solver': ['adam', 'sgd'],  # 优化器选项
    'alpha': [0.0001, 0.001, 0.01],  # 正则化参数选项
    'learning_rate': ['constant', 'adaptive']  # 学习率策略选项
}

In [7]:
ann = MLPRegressor(random_state=10)
model = GridSearchCV(estimator =ann,scoring='neg_root_mean_squared_error', param_grid = param_grid,cv=5, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train_stand, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('Bg_ANN13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 5 folds for each of 96 candidates, totalling 480 fits
{'mean_fit_time': array([0.17058554, 0.21773577, 0.23443518, 0.2228807 , 0.38931794,
       0.41954856, 0.52148762, 0.43859611, 0.48050547, 0.39778333,
       0.40480919, 0.43143101, 0.67468143, 0.60926862, 0.64328475,
       0.62767806, 0.28316751, 0.25671144, 0.29091063, 0.24771833,
       0.42033181, 0.365837  , 0.40884056, 0.38683658, 0.41551399,
       0.46633363, 0.34851165, 0.45276985, 0.65887299, 0.61715379,
       0.72151899, 0.61019382, 0.28699951, 0.27217178, 0.27641883,
       0.23524666, 0.39219732, 0.38463583, 0.44219446, 0.37552485,
       0.39050102, 0.44350839, 0.46791773, 0.56929054, 1.11662779,
       0.96793933, 1.08275633, 0.94982705, 0.53162379, 0.48804255,
       0.53853784, 0.49492049, 0.76299353, 0.63123174, 0.80421252,
       0.7403367 , 0.88052363, 0.79178634, 0.91832032, 0.81297526,
       1.1569418 , 0.94286709, 1.24717498, 1.18858428, 0.58732677,
       0.49931655, 0.58403254, 0.51843429, 0.8402

D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
